In [ ]:
!unzip -qq 'archive.zip' -d 'kaggle'

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM, Masking, GRU
from tensorflow.keras import Sequential

In [ ]:
DATAPATH = Path("kaggle")

PAD = 64.0

In [ ]:
# 6-CLASS CLASSIFICATION

# X = []
# y = []

# label_num = 0
# labels = {}

# for dir in os.listdir(DATAPATH):
#   labels[dir] = label_num
#   for file in os.listdir(DATAPATH / dir):
#     df = pd.read_csv(DATAPATH / dir / file, sep=';')
    
#     acc_x = df['AccelerationX'].to_numpy()
#     acc_y = df['AccelerationY'].to_numpy()
#     acc_z = df['AccelerationZ'].to_numpy()

#     X.append(np.dstack([acc_x, acc_y, acc_z]))
#     y.append(label_num)
    
#   label_num += 1

# y = np.array(y)

# max_len = max([x.shape[1] for x in X])
# X_padded = [tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=max_len, padding='post', value=PAD) for x in X]

# X_padded = np.float32(np.concatenate(X_padded))

# print(labels)

In [ ]:
# BINARY CLASSIFICATION

X = []
y = []

for dir in os.listdir(DATAPATH):
  
  label = 1 if 'Fall' in dir else 0

  for file in os.listdir(DATAPATH / dir):
    df = pd.read_csv(DATAPATH / dir / file, sep=';')
    
    acc_x = df['AccelerationX'].to_numpy()
    acc_y = df['AccelerationY'].to_numpy()
    acc_z = df['AccelerationZ'].to_numpy()

    X.append(np.dstack([acc_x, acc_y, acc_z]))
    y.append(label)

y = np.array(y)

max_len = max([x.shape[1] for x in X])
X_padded = [tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=max_len, padding='post', value=PAD) for x in X]
X_padded = np.float32(np.concatenate(X_padded))

In [ ]:
print(X_padded.shape)
print(y.shape)

(203, 1552, 3)
(203,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.1, stratify=y)

# X_train, X_val_test, y_train, y_val_test = train_test_split(X_padded, y, test_size=0.2, stratify=y)
# X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, stratify=y_val_test)

y_train = tf.keras.utils.to_categorical(y_train)
# y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# print(f'X_train: {X_train.shape}\ny_train: {y_train.shape}\nX_val:{X_val.shape}\ny_val:{y_val.shape}\nX_test: {X_test.shape}\ny_test: {y_test.shape}')

In [ ]:
print(f'X_train: {X_train.shape}\ny_train: {y_train.shape}\nX_test: {X_test.shape}\ny_test: {y_test.shape}')

X_train: (182, 1552, 3)
y_train: (182, 2)
X_test: (21, 1552, 3)
y_test: (21, 2)


In [ ]:
def create_model():
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  print(n_timesteps, n_features, n_outputs)
  model = Sequential(
      [
       Masking(mask_value=PAD, input_shape=(1552, 3)),
       GRU(128),
       Dropout(0.3),
       Dense(128, activation='relu'),
       Dropout(0.3),
       Dense(64, activation='relu'),
       Dense(2, activation='softmax')
      ]
  )
  
  opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model
  

In [ ]:
model1 = create_model()

1552 3 2


In [ ]:
model1(np.expand_dims(X_train[0], axis=0))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.5123726 , 0.48762745]], dtype=float32)>

In [ ]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

checkpoint_filepath = 'checkpoint'

!rm -rf $checkpoint_filepath

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq=110
)

model1.fit(X_train, y_train, batch_size=16, epochs=50,
           callbacks=[model_checkpoint_callback])

Epoch 1/50
12/12 [==============================] - 19s 1s/step - loss: 0.6901 - accuracy: 0.5275
Epoch 2/50
12/12 [==============================] - 14s 1s/step - loss: 0.6816 - accuracy: 0.5385
Epoch 3/50
12/12 [==============================] - 14s 1s/step - loss: 0.6725 - accuracy: 0.5769
Epoch 4/50
12/12 [==============================] - 14s 1s/step - loss: 0.6670 - accuracy: 0.6209
Epoch 5/50
12/12 [==============================] - 14s 1s/step - loss: 0.6632 - accuracy: 0.6209
Epoch 6/50
12/12 [==============================] - 14s 1s/step - loss: 0.6602 - accuracy: 0.6154
Epoch 7/50
12/12 [==============================] - 14s 1s/step - loss: 0.6589 - accuracy: 0.6264
Epoch 8/50
12/12 [==============================] - 14s 1s/step - loss: 0.6630 - accuracy: 0.6099
Epoch 9/50
12/12 [==============================] - 14s 1s/step - loss: 0.6591 - accuracy: 0.6209
Epoch 10/50
 1/12 [=>............................] - ETA: 12s - loss: 0.6181 - accuracy: 0.6250INFO:tensorflow:Assets 

INFO:tensorflow:Assets written to: checkpoint/assets


12/12 [==============================] - 21s 2s/step - loss: 0.6582 - accuracy: 0.6319
Epoch 11/50
12/12 [==============================] - 14s 1s/step - loss: 0.6466 - accuracy: 0.6209
Epoch 12/50
12/12 [==============================] - 14s 1s/step - loss: 0.6569 - accuracy: 0.6264
Epoch 13/50
12/12 [==============================] - 14s 1s/step - loss: 0.6516 - accuracy: 0.6209
Epoch 14/50
12/12 [==============================] - 14s 1s/step - loss: 0.6515 - accuracy: 0.6264
Epoch 15/50
12/12 [==============================] - 14s 1s/step - loss: 0.6524 - accuracy: 0.6154
Epoch 16/50
12/12 [==============================] - 14s 1s/step - loss: 0.6519 - accuracy: 0.6264
Epoch 17/50
12/12 [==============================] - 14s 1s/step - loss: 0.6592 - accuracy: 0.6264
Epoch 18/50
12/12 [==============================] - 14s 1s/step - loss: 0.6484 - accuracy: 0.6264
Epoch 19/50
 3/12 [======>.......................] - ETA: 10s - loss: 0.6866 - accuracy: 0.5625INFO:tensorflow:Assets wri

INFO:tensorflow:Assets written to: checkpoint/assets


12/12 [==============================] - 21s 2s/step - loss: 0.6516 - accuracy: 0.6374
Epoch 20/50
12/12 [==============================] - 14s 1s/step - loss: 0.6447 - accuracy: 0.6209
Epoch 21/50
12/12 [==============================] - 14s 1s/step - loss: 0.6411 - accuracy: 0.6319
Epoch 22/50
12/12 [==============================] - 14s 1s/step - loss: 0.6448 - accuracy: 0.6374
Epoch 23/50
12/12 [==============================] - 14s 1s/step - loss: 0.6446 - accuracy: 0.6264
Epoch 24/50
12/12 [==============================] - 14s 1s/step - loss: 0.6448 - accuracy: 0.6154
Epoch 25/50
12/12 [==============================] - 14s 1s/step - loss: 0.6511 - accuracy: 0.6044
Epoch 26/50
12/12 [==============================] - 14s 1s/step - loss: 0.6386 - accuracy: 0.6593
Epoch 27/50
12/12 [==============================] - 14s 1s/step - loss: 0.6453 - accuracy: 0.6154
Epoch 28/50
 5/12 [===========>..................] - ETA: 8s - loss: 0.5735 - accuracy: 0.7375INFO:tensorflow:Assets writ

...

12/12 [==============================] - 22s 2s/step - loss: 0.6428 - accuracy: 0.6154
Epoch 29/50
12/12 [==============================] - 14s 1s/step - loss: 0.6386 - accuracy: 0.6374
Epoch 30/50
12/12 [==============================] - 14s 1s/step - loss: 0.6304 - accuracy: 0.6538
Epoch 31/50
12/12 [==============================] - 14s 1s/step - loss: 0.6463 - accuracy: 0.5824
Epoch 32/50
12/12 [==============================] - 14s 1s/step - loss: 0.6330 - accuracy: 0.6154
Epoch 33/50
12/12 [==============================] - 14s 1s/step - loss: 0.6377 - accuracy: 0.6044
Epoch 34/50
12/12 [==============================] - 14s 1s/step - loss: 0.6352 - accuracy: 0.6538
Epoch 35/50
12/12 [==============================] - 14s 1s/step - loss: 0.6331 - accuracy: 0.6374
Epoch 36/50
12/12 [==============================] - 14s 1s/step - loss: 0.6373 - accuracy: 0.6154
Epoch 37/50
 7/12 [================>.............] - ETA: 5s - loss: 0.6071 - accuracy: 0.6518INFO:tensorflow:Assets writ

INFO:tensorflow:Assets written to: checkpoint/assets


12/12 [==============================] - 22s 2s/step - loss: 0.6341 - accuracy: 0.6154
Epoch 38/50
12/12 [==============================] - 14s 1s/step - loss: 0.6154 - accuracy: 0.6429
Epoch 39/50
12/12 [==============================] - 14s 1s/step - loss: 0.6362 - accuracy: 0.6264
Epoch 40/50
12/12 [==============================] - 14s 1s/step - loss: 0.6305 - accuracy: 0.6429
Epoch 41/50
12/12 [==============================] - 14s 1s/step - loss: 0.6248 - accuracy: 0.6429
Epoch 42/50
12/12 [==============================] - 14s 1s/step - loss: 0.6314 - accuracy: 0.6648
Epoch 43/50
12/12 [==============================] - 14s 1s/step - loss: 0.6224 - accuracy: 0.6154
Epoch 44/50
12/12 [==============================] - 14s 1s/step - loss: 0.6106 - accuracy: 0.6429
Epoch 45/50
12/12 [==============================] - 14s 1s/step - loss: 0.6201 - accuracy: 0.6538
Epoch 46/50
 9/12 [=====================>........] - ETA: 3s - loss: 0.6658 - accuracy: 0.6250INFO:tensorflow:Assets writ

INFO:tensorflow:Assets written to: checkpoint/assets


12/12 [==============================] - 21s 2s/step - loss: 0.6370 - accuracy: 0.6538
Epoch 47/50
12/12 [==============================] - 14s 1s/step - loss: 0.6291 - accuracy: 0.6099
Epoch 48/50
12/12 [==============================] - 14s 1s/step - loss: 0.6284 - accuracy: 0.6429
Epoch 49/50
12/12 [==============================] - 14s 1s/step - loss: 0.6099 - accuracy: 0.6758
Epoch 50/50
12/12 [==============================] - 14s 1s/step - loss: 0.6107 - accuracy: 0.6758


In [ ]:
model1.evaluate(X_test, y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.8629 - accuracy: 0.6190


[0.8629382848739624, 0.6190476417541504]

In [ ]:
!mkdir final_model4
model1.save("final_model4/", save_format='tf')

!tar -cvzf weights.tar.gz final_model4

INFO:tensorflow:Assets written to: final_model4/assets


INFO:tensorflow:Assets written to: final_model4/assets


final_model4/
final_model4/variables/
final_model4/variables/variables.data-00000-of-00001
final_model4/variables/variables.index
final_model4/saved_model.pb
final_model4/keras_metadata.pb
final_model4/assets/


In [ ]:
!tar -xvf weights.tar.gz

final_weights/
final_weights/.index
final_weights/.data-00000-of-00001
final_weights/checkpoint


In [ ]:
# LOAD TESTS

In [ ]:
model2 = create_model()
model2.load_weights("final_weights/")


1552 3 2


In [ ]:
model1.get_weights()[0].dtype

dtype('float32')

In [ ]:
from keras import backend as K

In [ ]:
import tensorflow.compat.v2 as tf_comp
tf_comp.enable_v2_behavior()

In [ ]:
# K.set_floatx('float32')
model2 = create_model()
model2.load_weights("final_weights/")

1552 3 2


In [ ]:
export_dir = 'final_model'
model2.save(export_dir)

INFO:tensorflow:Assets written to: final_model/assets


INFO:tensorflow:Assets written to: final_model/assets


In [ ]:
model2.evaluate(X_test, y_test)

1/1 [==============================] - 5s 5s/step - loss: 0.0905 - accuracy: 1.0000


[0.09045404195785522, 1.0]

In [ ]:
# TODO: checkpoints corrupted

# model3 = tf.saved_model.load("checkpoint")
# model3.evaluate(X_test, y_test)

AttributeError: ignored

In [ ]:
model2 = create_model()
model2.load_weights("final_weights/")

1552 3 2


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("final_model4")
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS] 
converter._experimental_lower_tensor_list_ops = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
with open('fallDetectionModel.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# funkcja testujaca model z wykorzystaniem interpretera TFLite
# TODO zmodyfikuj funkcje tak by zwracala (accuracy, latency)
import time 
def test_model_accuracy(tflite_model):
  # Load TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path=tflite_model)
  interpreter.allocate_tensors()

  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]
  proper = 0
  start = time.time()
  
  for img, label in zip(X_test, y_test):
    interpreter.set_tensor(input_index, [img])

    interpreter.invoke()

    prediction = interpreter.get_tensor(output_index)
    label_is = np.argmax(prediction, axis=-1)[0]
    label_should_be = label
    if label_is == label_should_be:
      proper += 1
    #print("Label is {} should be {}.".format(label_is, label_should_be))

  end = time.time()
  amount = len(X_test)
  latency = (end-start)/amount
  accuracy = proper / amount
  return (accuracy, latency)


In [ ]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")

In [ ]:
interpreter.allocate_tensors()

In [ ]:
input_index = interpreter.get_input_details()[0]["index"]
input_index

0

In [ ]:
interpreter.get_input_details()[0]

{'dtype': numpy.float32,
 'index': 0,
 'name': 'serving_default_masking_3_input:0',
 'quantization': (0.0, 0),
 'quantization_parameters': {'quantized_dimension': 0,
  'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32)},
 'shape': array([   1, 1552,    3], dtype=int32),
 'shape_signature': array([  -1, 1552,    3], dtype=int32),
 'sparsity_parameters': {}}

In [ ]:
output_index = interpreter.get_output_details()[0]["index"]
output_index

79

In [ ]:
interpreter.set_tensor(input_index, [X_test[0]])

In [ ]:
X_test[0].dtype

dtype('float32')

In [ ]:
interpreter.invoke()

In [ ]:
prediction = interpreter.get_tensor(output_index)

In [ ]:
label_is = np.argmax(prediction, axis=-1)[0]

In [ ]:
prediction

array([[0.51676404, 0.48323593]], dtype=float32)

In [ ]:
label_is

0